In [153]:
import torch
import random
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import datasets
import os
from torchvision.transforms import ToTensor
from torchvision.io import read_image
import matplotlib.pyplot as plt
from torch import nn
from torch import optim
from tqdm import trange
import matplotlib.pyplot as plt
if torch.backends.mps.is_available():
    print("MPS is available on this device.")
    # Set the PyTorch device to MPS
    device = torch.device("cpu")

MPS is available on this device.


In [154]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, transform=None, target_transform=None):
        self.chunked = pd.read_csv(annotations_file, chunksize=60000)
        self.img_labels = next(self.chunked)
        self.transform = transform
        
    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        label = torch.tensor(self.img_labels.iloc[idx, :1].values, device = device)
        image = torch.tensor(self.img_labels.iloc[idx, 1:].values, dtype=torch.float, device = device)
        return image, label


class Mnet(nn.Module):
    def __init__(self):
        super(Mnet, self).__init__()
        self.l1 = nn.Linear(784, 128)
        self.act = nn.ReLU()
        self.l2 = nn.Linear(128, 30)
        self.l3 = nn.Linear(30,10)
    def forward(self, x):
        x = self.l1(x)
        x = self.act(x)
        x = self.l2(x)
        x = self.act(x)
        x = self.l3(x)
        return x

def validate_epoch(model, dat):
    accs = [batch_accuracy(model(x),y) for x,y in dat]
    return round(torch.stack(accs).mean().item(), 4)

def batch_accuracy(x, y):
    preds = softmax(x)
    predicted_value = torch.argmax(preds, dim=1)
    trgts = y.flatten()
    bools = predicted_value == trgts
    accs = bools.to(torch.float).mean()
    return accs

In [156]:
dl = CustomImageDataset("data/mnist_train.csv")
valid_dl = CustomImageDataset("data/mnist_test.csv")
train_data = DataLoader(dl, batch_size=128, shuffle=True)
test_data = DataLoader(valid_dl, batch_size=128, shuffle=True)
#model = nn.Sequential(nn.Linear(784, 128), nn.ReLU(), nn.Linear(128, 10), nn.ReLU())
model = Mnet().to(device)
opt = optim.Adam(model.parameters()) #optim.SGD(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

In [157]:
epochs = 10
losses, accuracies = [], [] 
for i in (r := trange(epochs)):
    for x, y in train_data:
        p = model(x)
        loss = loss_fn(p, y.flatten())
        losses.append(loss)
        loss.backward()
        opt.step()
        opt.zero_grad()
    accuracy = validate_epoch(model, test_data)
    accuracies.append(accuracy)
    r.set_description(f"accuracy: {accuracy}, loss: {loss.data}")

accuracy: 0.9742, loss: 0.006378551479429007: 100%|████████████████████████████████████████████████████████████████████████| 10/10 [00:36<00:00,  3.67s/it]


In [123]:
plt.plot(losses)

torch.Size([128, 784])